# Formatting previously scraped data

- Doing so on seperate doc to make looking at analysis easier

In [1]:
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
#explore data
df = pd.read_csv('marathons_all_no_index.csv')
print(len(df))
print(df.head(10))

12205
                                             0                      1  \
0                 Christmas Marathon - Results        Olympia, WA USA   
1              Jacksonville Marathon - Results   Jacksonville, FL USA   
2         Dallas White Rock Marathon - Results         Dallas, TX USA   
3                  Honolulu Marathon - Results       Honololu, HI USA   
4           Hops Marathon by the Bay - Results          Tampa, FL USA   
5             Kiawah Island Marathon - Results  Kiawah Island, SC USA   
6               Rocket City Marathon - Results     Hunstville, AL USA   
7  California International Marathon - Results     Sacramento, CA USA   
8                   Memphis Marathon - Results        Memphis, TN USA   
9                   Raleigh Marathon - Results        Raleigh, NC USA   

                   2                                                 3  \
0  December 17, 2000    Male Winner: 2:49:35 | Female Winner: 3:18:51    
1  December 16, 2000       Finishers: 683,

In [3]:
#columns for edited data
columns = ['name', 'year', 'month', 'day', 'date', 'location', 'male_winner', 'female_winner', 'finishers', 'male_finishers',
           'female_finishers', 'avg_finish', 'std']

df_edited = pd.DataFrame(columns = columns)
#easy columns to sort
df_edited['name'] = df['0'].str.rstrip(' - Results').str.lower()
df_edited['location'] = df['1']
df_edited['date'] = df['2']

#finishers columns: first pull finishers from original df
#then pull males and include 'males - ' so it doesn't match total finishers number 
#and then extract just number, do the same for females, than total finishers last
finishers_3 = df['3'].str.contains('Finishers')
df_edited['finishers'] = df['3'].where(finishers_3, other = '')
df_edited['male_finishers'] = df_edited['finishers'].str.extract(r'(Males - [0-9]+)')
df_edited['male_finishers'] = df_edited['male_finishers'].str.extract(r'([0-9]+)')
df_edited['female_finishers'] = df_edited['finishers'].str.extract(r'(Females - [0-9]+)')
df_edited['female_finishers'] = df_edited['female_finishers'].str.extract(r'([0-9]+)')
df_edited['finishers'] = df_edited['finishers'].str.extract(r'([0-9]+)')

#create month and day columns
df_edited['date'] = df_edited['date'].astype('datetime64[ns]')
df_edited['year'] = pd.DatetimeIndex(df_edited['date']).year
df_edited['month'] = pd.DatetimeIndex(df_edited['date']).month
df_edited['day'] = pd.DatetimeIndex(df_edited['date']).day

#average finish time and std
average_5 = df['5'].str.contains('Average Finish')
average_4 = df['4'].str.contains('Average Finish')
average_4_vals = df['4'].where(average_4)
df_edited['avg_finish'] = df['5'].where(average_5, other = average_4_vals)
df_edited['std'] = df_edited['avg_finish'].str.extract(r'(STD: [0-9]:[0-9]{2}:[0-9]{2})')
df_edited['std'] = df_edited['std'].str.slice(start = 5)
df_edited['avg_finish'] = df_edited['avg_finish'].str.extract(r'([0-9]{1,2}:[0-9]{2}:[0-9]{2})')

#male and female winners
winners_4 = df['4'].str.contains('Male Winner')
winners_3 = df['3'].str.contains('Male Winner')
winners_3_vals = df['3'].where(winners_3)
df_edited['male_winner'] = df['4'].where(winners_4, other = winners_3_vals)
df_edited['female_winner'] = df_edited['male_winner'].str.extract(r'(Female Winner: [0-9]{1,2}:[0-9]{2}:[0-9]{2})')
df_edited['female_winner'] = df_edited['female_winner'].str.extract(r'([0-9]{1,2}:[0-9]{2}:[0-9]{2})')
df_edited['male_winner'] = df_edited['male_winner'].str.extract(r'([0-9]{1,2}:[0-9]{2}:[0-9]{2})')
                                                 
#column type mapping
col_mapping = {'finishers': float, 'male_finishers': float, 'female_finishers': float
              }
print(df_edited.dtypes)
print(df_edited.head(10))

df_edited.to_csv('marathons_clean.csv', index = False)             

name                         object
year                          int64
month                         int64
day                           int64
date                 datetime64[ns]
location                     object
male_winner         timedelta64[ns]
female_winner       timedelta64[ns]
finishers                   float64
male_finishers              float64
female_finishers            float64
avg_finish          timedelta64[ns]
std                 timedelta64[ns]
dtype: object
                                name  year  month  day       date  \
0                 christmas marathon  2000     12   17 2000-12-17   
1              jacksonville marathon  2000     12   16 2000-12-16   
2         dallas white rock marathon  2000     12   10 2000-12-10   
3                  honolulu marathon  2000     12   10 2000-12-10   
4           hops marathon by the bay  2000     12   10 2000-12-10   
5             kiawah island marathon  2000     12    9 2000-12-09   
6               rocket city maratho